# Introduction

Author: Wenjie Tan
Andrew ID: wenjiet
Date: 2016/10/31

In this tutorial, we will build a distributed retrieval system, and have an exploration about building index on inverted list by map reduce, calculating scores based on tf-idf, improving precision and recall rate based on BM25.

## 1. Map Reduce Introduction

MapReduce is a programming model and an associated implementation for processing and generating large data sets with a parallel, distributed algorithm on a cluster. Conceptually similar approaches have been very well known since 1995 with the Message Passing Interface standard having reduce and scatter operations.

--from Wikipedia

Today, we will use Map Reduce to process the raw text, then we will get the tf and idf for future reference. This job will be done in Amazon Web Service using Elastic Map Reduce Cluster.

First, we need to have a familiar with MapReduce on AWS. The basic architecture of MapReduce on AWS is a pipeline environment. We need to write two python scripts, each describing how map and reduce works, and upload them into the AWS S3. After that we will need to configure an EMR, choose the mapper and reducer as what we have just uploaded. Then we specific the input directory and output directory, and then launch the cluster. Once in a while we finish the map reduce job, we will get all the reduce output in the output directory.

### 1. Map Reduce Sample Word Count

First of all, we need to be familiar with the usage of EMR on AWS. This sample we need to write a mapper and a reducer to read a lot of raw files, and then output the the count number of different word.

In Map reduce phrase, first we need a mapper and a reducer. Mapper will accept a lot of input, parsing them into (key, value) pair, and the system will do a shuffle sort based on key. Then the result will be sent to reducer as the input of the reducer. The reducer will parse these input and output the final (key, value) result.

For Mapper, it should read from the standard input, get each word of the text file, and output the word with "word\tcount", here count should be 1.

For Reducer, it should read from the standard input, concate the same word and add the count together, then output the word with "word\tcount". Note, word should have been sorted in the shuffle state.

Mapper:

In [ ]:
#!/usr/bin/env python
import sys

def word_count_mapper_main():
    line = sys.stdin.readline()
    try:
        while line:
            words = line.strip().split(" ")
            for word in words:
                print "%s\t%s" % (word, 1)
            line = sys.stdin.readline()
    except "end of file":
        return None

if __name__ == "__main__":
    word_count_mapper_main()

Reducer:

In [ ]:
#!/usr/bin/env python
import sys

def word_count_reducer_main():
    current_word = None
    current_count = 0
    word = None
    line = sys.stdin.readline()
    try:
        while line:
            word, count = line.split("\t")
            try:
                count = int(count)
            except ValueError:
                continue
            if current_word == word:
                current_count += count
            else:
                if current_word:
                    print "%s\t%s" % (current_word, current_count)
                current_count = count
                current_word = word
            line = sys.stdin.readline()
    except "end of file":
        if current_word == word:
            print "%s\t%s" % (current_word, current_count)

if __name__ == "__main__":
    word_count_reducer_main()

Then, we can simply test our mapper and reducer locally with the pipeline:

In [ ]:
cat sample_doc_0.txt | python mapper.py | sort | python reducer.py > word_count_result.txt

### 2. Get the Inverted Index

Inverted List is a really useful tool for document retrieval. 

An inverted index catalogs a collection of objects in their textual representations. Given a set of documents, keywords and other attributes (possibly including relevance ranking) are assigned to each document. The inverted index is the list of keywords and links to the corresponding document. Frequently there are several restrictions which limit the keywords in an index. 

A collection of stopwords--keywords that are not considered relevant. This collection normally contains words considered too common to function as keywords (articles, prepositions, conjunctions, etc.) or words outside the context of the index.

A set of rules that define keywords in a document. This controls the manner in which keywords are found. For instance, keywords might be defined as character sequences surrounded by white- space.

A set of rules that restrict indexable words. For instance, a rule that causes keywords containing numbers not to be indexed.

A collection of "synonyms"--keywords that should be indexed using a different keyword.
The InvertedIndex module provides simple tools for creating and maintaining inverted indices with support for incremental indexing and for stopword, synonym and stemming databases.

--from http://legacy.python.org/workshops/1996-11/papers/InvertedIndex.html

Here, we still try to use mapreduce to produce the inverted index by EMR.

The input is the same as above, each file with a name as "doc_1", "doc_2", ..., "doc_n", we need to process the text, and output the inverted index like:

school doc_1 doc_13 doc_203 ...
time doc_13 doc_14 doc_150 ...
....

In AWS EMR, we could use the environment variable 'map_input_file' to get the file name of the input document. And in our local run, we only need to set the environment varuable 'map_input_file' in our test script.

Mapper:

In [ ]:
#!/usr/bin/env python
import sys
import os

def word_count_mapper_main():
    line = sys.stdin.readline()
    try:
        while line:
            words = line.strip().split(" ")
            for word in words:
                print "%s\t%s" % (word, os.environ["map_input_file"])
            line = sys.stdin.readline()
    except "end of file":
        return None

if __name__ == "__main__":
    word_count_mapper_main()

Reducer:

In [ ]:
#!/usr/bin/env python
from __future__ import print_function
import sys

def output_word(word, doc):
    print(word, end = "")
    for d in doc:
        print(" " + d, end = "")
    print("")

def word_count_reducer_main():
    current_word = None
    current_doc = None
    all_doc = []
    line = sys.stdin.readline()

    while line:
        word, doc = line.strip().split("\t")
        if current_word == word:
            if current_doc != doc:
                all_doc.append(doc)
                current_doc = doc
        else:
            if current_word:
                output_word(current_word, all_doc)
            current_word = word
            current_doc = doc
            all_doc = []
            all_doc.append(doc)
        line = sys.stdin.readline()

    if current_word:
        output_word(current_word, all_doc)

if __name__ == "__main__":
    word_count_reducer_main()

Then, we can simply test our mapper and reducer locally with the pipeline:

In [ ]:
export map_input_file="sample_doc_0.txt"
cat sample_doc_0.txt | python mapper.py > mapper_output0.txt
export map_input_file="sample_doc_1.txt"
cat sample_doc_1.txt | python mapper.py > mapper_output1.txt
cat mapper_output* | sort | python reducer.py > inverted_list_result.txt
rm mapper_output*

### 3. Boolean Retrieval Model

The Boolean model of information retrieval (BIR)[1] is a classical information retrieval (IR) model and, at the same time, the first and most adopted one. It is used by many IR systems to this day.

--from https://en.wikipedia.org/wiki/Standard_Boolean_model

In Boolean Retrieval, we will have a query of a boolean logical formation, indicating the requirement of this query. For example, we have a query like:

"Carnegie" AND "Mellon" AND "University" AND "Data" AND "Science" AND "Course"

This query requires us to return all the documents which should contain all the words "Carnegie", "Mellon", ..., "Course".

Boolean Retrieval also could contain "Or", "Near", "Window" operator, howeverm here, we will try to achieve boolean retrieval model only containing "AND".

Remember, we already have the inverted list, and in the inverted list, we already have documents name sorted for each query word. As a result, it is simply that we use several pointers to all query words, move forward and judge. This way will save a lot of time. Also, pointer movements could be achieved by Priority Queue.

Now we actually have a retrieval system, which will take the inverted list as its argument, and take each line of the standard input as one query, parsing it, and output the result specific to this query.

Retrieval System: (Please see this model in file boolean_retrieve.py)

Then, we can simply test our mapper and reducer locally with the pipeline:

In [ ]:
cat qtest.txt | python boolean_retrieval.py inverted_list_result.txt > boolean_retrieval_result.txt

## 2. Use AWS EMR for large data processing

Now we have everything work well locally, and then we will put the script into AWS, and use EMR to generate the inverted index distributedly.

Actually, till now, everything is almost the same as we already have the Mapper and Reducer. The only thing we need to do is to set the input s3 directory, output s3 directory, mapper, reducer, machine number and type, and then lunch cluster. It is easier than you think.

### 1. Pre-processing

Before we continue on AWS EMR for large data, we need to do some filter job, i.e. stopwords remove, before the next step.

In typical English written document, there will be some words which occurred multiple times. Those words will consume a lot of space in retrieval model, and actually they have no meaning for us when searching. We call those words as stop_words, which means they need to be removed when building the index.

Besides, English words have different shapes and cases, also some punctuations. We need to have a clean through all those words, and change them into the same format we what.

In this part, we need to rewrite our Mapper.

Mapper:

In [ ]:
#!/usr/bin/env python
import sys
import os
import string

def word_count_mapper_main():
    stopwords = {""}
    file = open("stopwords.txt", "r")
    line = file.readline()
    while line:
        stopwords.add(line.strip())
        line = file.readline()
    
    line = sys.stdin.readline()
    while line:
        text = line.strip().lower().replace("'s", "").replace("'", "")
        for p in string.punctuation:
            text = text.replace(p, " ")
        for word in text.split(" "):
            if len(word) > 0 and word not in stopwords:
                print "%s\t%s" % (word, os.environ["map_input_file"])
        line = sys.stdin.readline()

if __name__ == "__main__":
    word_count_mapper_main()

### 2. Boolean retrieval Model on EMR

At first, we need to have an account at AWS. After registration, we could need to generate the key pair to access the cluster.

Then, we need to create a s3 bucket to store our documents as the input, our Mapper.py and Reducer.py and the mapper and reducer, and specific locations for logging and outputing.

In the end, we could use Boto to lunch the cluster:

In [ ]:
import boto.emr
from boto.emr.connection import EmrConnection
from boto.emr.step import StreamingStep

conn = boto.emr.connect_to_region('us-west-2')
step = StreamingStep(name='boolean retrieval index build',
                     mapper='s3n://<s3 bucket>/Mapper.py',
                     reducer='s3n://<s3 bucket>/Reducer.py',
                     input='s3n://<s3 bucket>/input',
                     output='s3n://<s3 bucket>/output')
jobid = conn.run_jobflow(name='index building job flow',
                         log_uri='s3://<s3 bucket>/logs',
                         step=[step])
conn.add_jobflow_steps(jobid, [second_step])

## 3. Advanced Retrieval Model

This part will introduce some advanced ways in document retrieval.

### 1. Boolean Retrieval with TF-IDF

In information retrieval, tf-idf, short for term frequency-inverse document frequency, is a numerical statistic that is intended to reflect how important a word is to a document in a collection or corpus. It is often used as a weighting factor in information retrieval and text mining. The tf-idf value increases proportionally to the number of times a word appears in the document, but is offset by the frequency of the word in the corpus, which helps to adjust for the fact that some words appear more frequently in general.

--from https://en.wikipedia.org/wiki/Tf%E2%80%93idf

The difference between boolean retrieval and TF-IDF way is that we need to add a score for each query and document pair. The calculation of this score is regard as:

Score(q, d) = tf(q, d) * idf(q)

Here, tf(q, d) means how many times query word q occurred in document d, and idf means how the inverse of the number of how many documents have the query word q.

IDF is useful unpon the intuitive idea that rare words should weight more. Here, we have a small change to IDF function:

idf(q) = log(N / n(q))

N is the total number of the documents, while n(q) is the number of documents containing word q. And then, we add the score for every word in q to calculate the final score.

Reducer:

In [ ]:
#!/usr/bin/env python
from __future__ import print_function
import sys

def output_word(word, doc):
    print(word, end = "")
    for d in doc:
        print(" " + d, end = "")
    print("")

def word_count_reducer_main():
    current_word = None
    current_doc = None
    current_num = 0
    all_doc = []
    line = sys.stdin.readline()

    while line:
        word, doc = line.strip().split("\t")
        if current_word == word:
            if current_doc != doc:
                all_doc.append(doc + "," + str(current_num))
                current_doc = doc
                current_num = 1
            else:
                current_num += 1
        else:
            if current_word:
                output_word(current_word, all_doc)
            current_word = word
            current_doc = doc
            current_num = 1
            all_doc = []
            all_doc.append(doc + "," + str(current_num))
        line = sys.stdin.readline()

    if current_word:
        output_word(current_word, all_doc)

if __name__ == "__main__":
    word_count_reducer_main()

Retrieval System: (please see this model in tf_idf_retrieve.py)

### 2. BM25

In information retrieval, Okapi BM25 (BM stands for Best Matching) is a ranking function used by search engines to rank matching documents according to their relevance to a given search query. It is based on the probabilistic retrieval framework developed in the 1970s and 1980s by Stephen E. Robertson, Karen Spärck Jones, and others.

--from https://en.wikipedia.org/wiki/Okapi_BM25

Actually, BM25 is a revised version of TFIDF. The only difference is the way to calculate the score of query and document.

Here is the formulation:

score(q, d) = idf(q) * tf(q, d) * (k1 + 1) / (tf(q, d) + k1 * (1 - b + b * |D| / avg_doc_len))

here, k1 and b are two parameters, and we set to k1=1.2, b=0.75;
|D| is the total number of the documents, while avg_doc_len is the average document length.

So we could get the final BM25 retrieval system.

Retrieval System:

In [ ]:
#!/usr/bin/env python
import sys
from Queue import PriorityQueue
import math

def bm25_retrieval(inverted_list, D, q_words):
    # get the total size of the documents
    N = len(reduce(lambda x, y: x | y,
               map(lambda i: set(map(lambda j: j.split(',')[0], i)),
                   inverted_list.values())))
    k1 = 1.2
    b = 0.75
    avg_doc_len = tot_doc_len / N
    words, ret = [], []
    for word in q_words:
        if word not in inverted_list:
            return ret
        words.append(word)
    n = len(words)

    # current index for each id
    idx = [0] * n

    # initialize priority queue
    # in PQ, (current_word, id) is stored
    # cur_max_word stores the maximum word in PQ
    PQ = PriorityQueue()
    cur_max_doc = ""
    for i in range(n):
        cur_doc = inverted_list[words[i]][0].split(',')[0]
        PQ.put((cur_doc, i))
        cur_max_doc = max(cur_max_doc, cur_doc)
    while True:
        cur_min_doc, id = PQ.get()
        if cur_min_doc == cur_max_doc:
            # calculate the sum of tf-idf score of all q_word
            sum_tfidf = 0.0
            for i in range(n):
                doc, num = inverted_list[words[i]][idx[i]].split(',')
                tf = float(num)
                df = len(inverted_list[words[i]])
                idf = math.log(1.0 * N / df)
                # change to BM25 score function
                sum_tfidf += idf * tf * (k1 + 1) / (tf + k1 * (1 - b + b * D[doc] / avg_doc_len))
            ret.append((sum_tfidf, cur_min_doc))
        idx[id] += 1
        if idx[id] >= len(inverted_list[words[id]]):
            # sort the result, and then output
            ret.sort(reverse=True)
            return ret
        next_doc = inverted_list[words[id]][idx[id]].split(',')[0]
        cur_max_doc = max(cur_max_doc, next_doc)
        PQ.put((next_doc, id))

if __name__ == "__main__":
    if len(sys.argv) < 2:
        print "inverted file name needed."
        exit()
    inverted_list = {}
    file = open(sys.argv[1], "r")
    line = file.readline()
    while line:
        parts = line.strip().split(" ")
        inverted_list[parts[0]] = parts[1:]
        line = file.readline()

    D = {}  # store (doc, len)
    tot_doc_len = 0.0
    for word_value in inverted_list.values():
        for value in word_value:
            doc, num = value.split(',')
            if doc not in D:
                D[doc] = 0.0
            D[doc] += float(num)
            tot_doc_len += float(num)

    q_line = sys.stdin.readline()
    while q_line:
        q_words = q_line.strip().split(" ")
        doc = bm25_retrieval(inverted_list, D, q_words)
        print doc
        q_line = sys.stdin.readline()